In [1]:
import numpy as np
from bitstring import Bits

In [106]:
def convert_int_to_binary(number, bit_width):
	if((bit_width%4) != 0):
		return str(Bits(int=number, length=bit_width))
	else:
		temp =  str(Bits(int=number, length=bit_width+1))
		return '0b' + temp[-(len(temp)-3):]

def convert_binary_to_hex(number, bit_width):
	if(bit_width <= 4):
		return hex(int(number, 2))
	else:
		number = number[2:len(number)]
		data_ret = ''
		index = np.arange(bit_width, 1, -4)
		for i in range(len(index)):
			if(index[i] >= 4):
				temp = hex(int(number[index[i] - 4 : index[i]], 2))
				data_ret = temp[-1:] + data_ret
			else:
				temp = hex(int(number[0 : index[i]], 2))
				data_ret = temp[-1:] + data_ret
		
		return '0x' + data_ret

def create_txt_file_for_w_verilog(data_matrix, num_mutiplier_parallel, bit_width, file_path):
	verilog_file = open(file_path, mode = 'w')

	num_node_next_layer, num_node_previous_layer = data_matrix.shape

	num_sub_matrix = round(num_node_next_layer/num_mutiplier_parallel)
	row_sub_matrix = num_mutiplier_parallel
	col_sub_matrix = num_node_previous_layer

	for i in range(num_sub_matrix):
		sub_matrix = data_matrix[num_mutiplier_parallel*i:num_mutiplier_parallel*(i+1), :]
		for j in range(col_sub_matrix):
			bin_final = ''
			for k in range(row_sub_matrix):
				dec_data = int(sub_matrix[k][j])
				bin_data_temp = convert_int_to_binary(dec_data, bit_width)
				bin_data = bin_data_temp[-bit_width:]
				bin_final = bin_final + bin_data

			sub_bin_final_0 = convert_binary_to_hex('0b' + bin_final[0:round(len(bin_final)/2)], int(bit_width*row_sub_matrix/2))
			sub_bin_final_1 = convert_binary_to_hex('0b' + bin_final[-round(len(bin_final)/2):], int(bit_width*row_sub_matrix/2))

			if((j == (col_sub_matrix - 1)) and (i == num_sub_matrix - 1)):
				verilog_file.write(sub_bin_final_0[-(len(sub_bin_final_0)-2):] + ' ' + sub_bin_final_1[-(len(sub_bin_final_1)-2):])
			else:
				verilog_file.write(sub_bin_final_0[-(len(sub_bin_final_0)-2):] + ' ' + sub_bin_final_1[-(len(sub_bin_final_1)-2):] + '\n')
	verilog_file.close()

def create_txt_file_for_bias_verilog(data_matrix, num_mutiplier_parallel, bit_width, file_path):
	verilog_file = open(file_path, mode = 'w')

	data_matrix = data_matrix.reshape(-1, 1)

	num_node_next_layer, temp = data_matrix.shape

	num_sub_matrix = round(num_node_next_layer/num_mutiplier_parallel)
	len_sub_matrix = num_mutiplier_parallel
	for i in range(num_sub_matrix):
		sub_matrix = data_matrix[num_mutiplier_parallel*i:num_mutiplier_parallel*(i+1)]
		bin_final = ''
		for j in range(len_sub_matrix):
			dec_data = int(sub_matrix[j])
			bin_data_temp = convert_int_to_binary(dec_data, bit_width)
			bin_data = bin_data_temp[-bit_width:]
			bin_final = bin_final + bin_data
		
		sub_bin_final_0 = convert_binary_to_hex('0b' + bin_final[0:round(len(bin_final)/2)], int(bit_width*len_sub_matrix/2))
		sub_bin_final_1 = convert_binary_to_hex('0b' + bin_final[-round(len(bin_final)/2):], int(bit_width*len_sub_matrix/2))

		if(i == num_sub_matrix - 1):
			verilog_file.write(sub_bin_final_0[-(len(sub_bin_final_0)-2):] + ' ' + sub_bin_final_1[-(len(sub_bin_final_1)-2):])
		else:
			verilog_file.write(sub_bin_final_0[-(len(sub_bin_final_0)-2):] + ' ' + sub_bin_final_1[-(len(sub_bin_final_1)-2):] + '\n')
	verilog_file.close()

In [12]:
# ============================ Variables  ============================
num_mutiplier_parallel_for_acc_0 = 120
num_mutiplier_parallel_for_acc_1 = 84
num_mutiplier_parallel_for_acc_2 = 2

bit_width_quantization_w1 = 8
max_q_w1 = 2**(bit_width_quantization_w1-1)-1
min_q_w1 = -2**(bit_width_quantization_w1-1)
w1_verilog_file_path = '../../Weight_and_bias_of_model_verilog/W1.txt'

bit_width_quantization_w2 = 4
max_q_w2 = 2**(bit_width_quantization_w2-1)-1
min_q_w2 = -2**(bit_width_quantization_w2-1)
w2_verilog_file_path = '../../Weight_and_bias_of_model_verilog/W2.txt'

bit_width_quantization_w3 = 4
max_q_w3 = 2**(bit_width_quantization_w3-1)-1
min_q_w3 = -2**(bit_width_quantization_w3-1)
w3_verilog_file_path = '../../Weight_and_bias_of_model_verilog/W3.txt'

bit_width_quantization_b1 = 8
max_q_b1 = 2**(bit_width_quantization_b1-1)-1
min_q_b1 = -2**(bit_width_quantization_b1-1)
b1_verilog_file_path = '../../Weight_and_bias_of_model_verilog/B1.txt'

bit_width_quantization_b2 = 4
max_q_b2 = 2**(bit_width_quantization_b2-1)-1
min_q_b2 = -2**(bit_width_quantization_b2-1)
b2_verilog_file_path = '../../Weight_and_bias_of_model_verilog/B2.txt'

bit_width_quantization_b3 = 4
max_q_b3 = 2**(bit_width_quantization_b3-1)-1
min_q_b3 = -2**(bit_width_quantization_b3-1)
b3_verilog_file_path = '../../Weight_and_bias_of_model_verilog/B3.txt'

In [103]:
data_matrix = np.array([[ -30,  -62,  116],
[  90, -106,    1],
[ -48,   76,  -39],
[ -56,   45,   60]])
"""
data_matrix = np.array([[ 0],
[ 1],
[10],
[19], [15], [16], [17], [18]])
"""
print(data_matrix)

[[ -30  -62  116]
 [  90 -106    1]
 [ -48   76  -39]
 [ -56   45   60]]


In [107]:
# =========================== Input ==========================
create_txt_file_for_bias_verilog(data_matrix, 2, 8, w1_verilog_file_path)

In [100]:
temp = convert_int_to_binary(1, 8)

temp_hex = hex(int(temp, 2))

temp_hex_new = convert_binary_to_hex('010', 3)

print(temp)
print(temp_hex)
print(temp_hex_new)

0b00000001
0x1
0x2


In [22]:
print(temp[-8:])

00000001


In [35]:
a = np.arange(12, 1, -4)
print(a)
print(len(a))
print(a[0:2])

[12  8  4]
3
[12  8]
